In [1]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)
    
from src.predictionModule.LoadupSamples import LoadupSamples
from src.predictionModule.FilterSamples import FilterSamples
from src.predictionModule.MachineModels import MachineModels

import numpy as np
import polars as pl
import pandas as pd
import datetime

import logging
formatted_date = datetime.datetime.now().strftime("%d%b%y_%H%M").lower()

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter(fmt="%(asctime)s - %(message)s")
handler.setFormatter(formatter)
if not logger.hasHandlers():
    logger.addHandler(handler)
else:
    logger.handlers[:] = [handler]

#Output File handler
formatted_str = f"notebook-FilterSamples-optuna-{formatted_date}"
file_handler = logging.FileHandler(f"{formatted_str}.log", mode="w")
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

# Usage
logger.info("This will print to the notebook's output cell")

2025-08-29 17:52:51,404 - This will print to the notebook's output cell


In [ ]:
params = {
    "idxAfterPrediction": 5,
    'timesteps': 60,
    'target_option': 'last',
    "LoadupSamples_time_scaling_stretch": True,
    "LoadupSamples_time_inc_factor": 61,
    
    "LSTM_units": 32,
    "LSTM_num_layers": 2,
    "LSTM_dropout": 0.006086,
    "LSTM_recurrent_dropout": 0.001341,
    "LSTM_learning_rate": 0.000195,
    "LSTM_optimizer": "adam",
    "LSTM_bidirectional": True,
    "LSTM_batch_size": 2**12,
    "LSTM_epochs": 4,
    "LSTM_l1": 0.000016,
    "LSTM_l2": 0.000121,
    "LSTM_inter_dropout": 0.022068,
    "LSTM_input_gaussian_noise": 0.001,
    "LSTM_conv1d": True,
    "LSTM_conv1d_kernel_size": 3,
    "LSTM_loss": "mse",

    "FilterSamples_q_up": 0.99,

    "FilterSamples_lincomb_epochs": 5,
    "FilterSamples_lincomb_show_progress": False,
    "FilterSamples_lincomb_featureratio": 0.5,
    "FilterSamples_lincomb_itermax": 1,
    "FilterSamples_lincomb_init_toprand":  3,
    "FilterSamples_lincomb_batch_size": 2**12,

    "FilterSamples_days_to_train_end": 115,
    "FilterSamples_cat_over20": True,
    "FilterSamples_cat_posOneYearReturn": False,
    "FilterSamples_cat_posFiveYearReturn": False,
    "FilterSamples_taylor_horizon_days": 50,
    "FilterSamples_taylor_roll_window_days": 10,
    "FilterSamples_taylor_weight_slope": 1.268923
}

In [3]:
timegroup = "group_regOHLCV_over5years"
treegroup = "group_finanTo2011"

eval_date = datetime.date(year=2025, month=7, day=13)
evaldates = [eval_date - datetime.timedelta(days=i) for i in range(1, 6)]
start_train_date = datetime.date(year=2020, month=1, day=1)
split_Date = datetime.date(year=2025, month=1, day=1)
ls = LoadupSamples(
    train_start_date=start_train_date,
    test_dates=evaldates,
    treegroup=treegroup,
    timegroup=timegroup,
    params=params,
)
ls.load_samples(main_path = "../src/featureAlchemy/bin/")
ls.split_dataset(
    start_date=start_train_date,
    last_train_date=split_Date,
    last_test_date=eval_date
)

2025-08-29 17:53:39,517 - Test date 2025-07-12 not found in the database. Omitting.
2025-08-29 17:53:45,275 - Non-finite/too-large values in train y tree: 15 samples.
2025-08-29 17:53:45,275 - Non-finite/too-large values in train X tree: 86 samples.
2025-08-29 17:53:45,282 - Non-finite/too-large values in train y time: 15 samples.
2025-08-29 17:53:45,283 - Removing 101 samples from training data.


In [4]:
fs_pre = FilterSamples(
    Xtree_train=ls.train_Xtree,
    ytree_train=ls.train_ytree,
    treenames=ls.featureTreeNames,
    Xtree_test=ls.test_Xtree,
    ytree_test=ls.test_ytree,
    meta_train=ls.meta_pl_train,
    meta_test=ls.meta_pl_test,
    params=params
)

cat_mask_train, cat_mask_test = fs_pre.categorical_masks()
ls.apply_masks(cat_mask_train, cat_mask_test)

In [5]:
"""
Runs LSTM to generate feature(s) to add to the tree data.
"""
mm = MachineModels(params)

device = 'cuda'
starttime = datetime.datetime.now()
lstm_model, res_dict = mm.run_LSTM_torch(
    ls.train_Xtime, 
    ls.train_ytime, 
    ls.test_Xtime, 
    ls.test_ytime, 
    device=device
)
preds_train = mm.predict_LSTM_torch(lstm_model, ls.train_Xtime, batch_size=params["LSTM_batch_size"], device=device)
preds_test = mm.predict_LSTM_torch(lstm_model, ls.test_Xtime, batch_size=params["LSTM_batch_size"], device=device)
endtime = datetime.datetime.now()

logger.info(f"  LSTM RSME: {res_dict['val_rmse']*2/params['LoadupSamples_time_inc_factor']:.4f}")
logger.info(f"  LSTM completed in {endtime - starttime}.")
for q in [0.95, 0.98, 0.99]:
    q_pred = np.quantile(preds_train, q)
    logger.info(f"  LSTM quantile {q:.2f} of train predictions: {q_pred:.4f}")
    filtered = ls.train_ytree[preds_train >= q_pred]
    logger.info(f"  Result of quantile {q:.2f} of train set (mean): {filtered.mean():.4f}")

## Add LSTM predictions to the tree test set
train_std = np.std(preds_train)
test_std = 1.0 if np.std(preds_test) < 1e-6 else np.std(preds_test)
ls.train_Xtree = np.hstack((ls.train_Xtree, ((preds_train-1.0)/train_std).reshape(-1, 1)))
ls.test_Xtree = np.hstack((ls.test_Xtree, ((preds_test-1.0)/test_std).reshape(-1, 1)))
if isinstance(ls.featureTreeNames, list):
    ls.featureTreeNames.append("LSTM_Prediction")
elif isinstance(ls.featureTreeNames, np.ndarray):
    ls.featureTreeNames = np.append(ls.featureTreeNames, "LSTM_Prediction")

Epochs:   0%|          | 0/4 [00:00<?, ?it/s]

2025-08-29 17:54:29,535 - Epoch 1/4 — Train RMSE: 0.4515 — Validation RMSE: 0.4070


Epochs:  25%|██▌       | 1/4 [00:14<00:44, 14.75s/it]

2025-08-29 17:54:43,934 - Epoch 2/4 — Train RMSE: 0.4100 — Validation RMSE: 0.4069


Epochs:  50%|█████     | 2/4 [00:29<00:29, 14.54s/it]

2025-08-29 17:54:58,449 - Epoch 3/4 — Train RMSE: 0.4099 — Validation RMSE: 0.4069


Epochs:  75%|███████▌  | 3/4 [00:43<00:14, 14.53s/it]

2025-08-29 17:55:12,877 - Epoch 4/4 — Train RMSE: 0.4098 — Validation RMSE: 0.4069


Epochs: 100%|██████████| 4/4 [00:58<00:00, 14.52s/it]


2025-08-29 17:55:18,706 -   LSTM RSME: 0.0133
2025-08-29 17:55:18,706 -   LSTM completed in 0:01:05.405004.
2025-08-29 17:55:18,716 -   LSTM quantile 0.95 of train predictions: 0.5437
2025-08-29 17:55:18,719 -   Result of quantile 0.95 of train set (mean): 1.0011
2025-08-29 17:55:18,728 -   LSTM quantile 0.98 of train predictions: 0.5506
2025-08-29 17:55:18,728 -   Result of quantile 0.98 of train set (mean): 0.9983
2025-08-29 17:55:18,739 -   LSTM quantile 0.99 of train predictions: 0.5549
2025-08-29 17:55:18,740 -   Result of quantile 0.99 of train set (mean): 0.9953


In [6]:
import optuna
logger.setLevel(logging.INFO)

n_reruns = 30
time_delta_days = 25
split_dates = [split_Date - datetime.timedelta(days=time_delta_days * i) for i in range(0, n_reruns)]
n_testdays = 7
studytime = 60*60*5
n_startup_trials = 15
objective_name = "taylor"
studyname = f"optuna_{formatted_str}_{objective_name}"
params["FilterSamples_q_up"] = 0.99

c:\Users\kimer\Desktop\RandomOdyssey\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
def sample_params(trial, space):
    out = {}
    for name, spec in space.items():
        kind, lo, hi, kw = spec
        sug = trial.suggest_int if kind == "int" else trial.suggest_float
        out[name] = sug(name.replace("FilterSamples_", ""), lo, hi, **kw)
    return out

def evaluate_objective(opt_params, mask_getter, trial_num):
    scores = []
    for sd in split_dates[:n_reruns]:
        lsc = ls.copy(deep=True)
        last_test_day = sd + datetime.timedelta(days=n_testdays)
        lsc.split_dataset(start_date=start_train_date, last_train_date=sd, last_test_date=last_test_day)

        fs = FilterSamples(
            Xtree_train=lsc.train_Xtree, ytree_train=lsc.train_ytree, treenames=lsc.featureTreeNames,
            Xtree_test=lsc.test_Xtree,  ytree_test=lsc.test_ytree,
            meta_train=lsc.meta_pl_train, meta_test=lsc.meta_pl_test, params=opt_params
        )
        mask_train, mask_test = mask_getter(fs)

        logger.info(f"Trial number {trial_num}")
        score_train = fs.evaluate_mask(mask_train, lsc.meta_pl_train['date'], lsc.train_ytree)
        score_test  = fs.evaluate_mask(mask_test,  lsc.meta_pl_test['date'],  lsc.test_ytree)
        logger.info(f"  Score (train) = {score_train}")
        logger.info(f"  Score (test)  = {score_test}")

        scores.append(score_test)

    s = np.array(scores, dtype=float)
    s[~np.isfinite(s)] = 1.0
    s[s <= 0] = 1.0
    return float(np.log(s).mean()) if s.size else float("-inf")

LINCOMB_SPACE = {
    "FilterSamples_days_to_train_end": ("int", 250, 350, {"step": 10}),
    "FilterSamples_lincomb_lr": ("float", 1e-4, 5e-3, {"log": False}),
    "FilterSamples_lincomb_epochs": ("int", 10, 200, {"step": 10}),
    "FilterSamples_lincomb_probs_noise_std": ("float", 0.1, 0.3, {"log": False}),
    "FilterSamples_lincomb_subsample_ratio": ("float", 0.2, 0.4, {}),
    "FilterSamples_lincomb_sharpness": ("float", 2.0, 5.0, {"log": False}),
    #"FilterSamples_lincomb_featureratio": ("float", 0.05, 0.99, {"log": True}),
    #"FilterSamples_lincomb_itermax": ("int", 1, 3, {}),
    "FilterSamples_lincomb_init_toprand": ("int", 2, 15, {})
}
TAYLOR_SPACE = {
    "FilterSamples_days_to_train_end": ("int", 5, 130, {"step": 5}),
    "FilterSamples_taylor_horizon_days": ("int", 5, 80, {"step": 5}),
    "FilterSamples_taylor_roll_window_days": ("int", 5, 80, {"step": 5}),
    "FilterSamples_taylor_weight_slope": ("float", 0.1, 1.5, {"log": False})
}

def make_objective(space, mask_getter, static_overrides=None):
    static_overrides = static_overrides or {}
    def objective(trial: optuna.Trial) -> float:
        opt_params = params.copy()
        opt_params.update(sample_params(trial, space))
        opt_params.update(static_overrides)
        return evaluate_objective(opt_params, mask_getter, trial.number)
    return objective

objective_lincomb = make_objective(
    LINCOMB_SPACE,
    lambda fs: fs.lincomb_masks(),
    {"FilterSamples_lincomb_show_progress": False},
)
objective_taylor = make_objective(
    TAYLOR_SPACE,
    lambda fs: fs.taylor_feature_masks(),
)

In [8]:
if objective_name == "lincomb":
    objective = objective_lincomb
if objective_name == "taylor":
    objective = objective_taylor

In [9]:
optuna.logging.enable_propagation()
sampler = optuna.samplers.TPESampler(n_startup_trials=n_startup_trials)
study = optuna.create_study(
    study_name = studyname,
    storage="sqlite:///sandbox_optuna.db",
    direction="maximize",
    load_if_exists=True,
    sampler=sampler,
)
study.optimize(objective, timeout=studytime)

logger.info(f"Best parameters: {study.best_params}")
logger.info(f"Best score: {study.best_value}")

df: pd.DataFrame = study.trials_dataframe()
logger.info("\nTrials DataFrame:")
logger.info(df.sort_values("value").to_string())

param_importances = optuna.importance.get_param_importances(study)
logger.info("Parameter Importances:")
for key, value in param_importances.items():
    logger.info(f"{key}: {value}")

[I 2025-08-29 17:55:20,446] A new study created in RDB with name: optuna_notebook-FilterSamples-optuna-29aug25_1752_taylor


2025-08-29 17:55:20,446 - A new study created in RDB with name: optuna_notebook-FilterSamples-optuna-29aug25_1752_taylor
2025-08-29 17:55:33,280 - FilterSamples/Taylor: (train) mean of y values 1.0006264340729107.
2025-08-29 17:55:33,284 - FilterSamples/Taylor: (test) mean of y values 1.0022854508909003.
2025-08-29 17:55:33,855 - FilterSamples/Taylor: Selected feature 'MathFeature_Price_logDiff' with score 1.478792 and threshold 1.9536 (q=0.99).
2025-08-29 17:55:34,670 - FilterSamples/Taylor: Final score (train): 0.9959
2025-08-29 17:55:34,672 - FilterSamples/Taylor: Final score (test): 1.0002
2025-08-29 17:55:34,672 - FilterSamples/Taylor: Fraction of days with no coverage by test mask: 0.00%
2025-08-29 17:55:34,676 - Trial number 0
2025-08-29 17:55:35,307 -   Score (train) = 0.9958607959105817
2025-08-29 17:55:35,307 -   Score (test)  = 1.000215422602711
2025-08-29 17:55:48,540 - FilterSamples/Taylor: (train) mean of y values 1.000824408082611.
2025-08-29 17:55:48,542 - FilterSamples

[I 2025-08-29 18:00:35,534] Trial 0 finished with value: 0.0026881472253545585 and parameters: {'days_to_train_end': 80, 'taylor_horizon_days': 60, 'taylor_roll_window_days': 5, 'taylor_weight_slope': 0.8321861754078372}. Best is trial 0 with value: 0.0026881472253545585.


2025-08-29 18:00:35,534 - Trial 0 finished with value: 0.0026881472253545585 and parameters: {'days_to_train_end': 80, 'taylor_horizon_days': 60, 'taylor_roll_window_days': 5, 'taylor_weight_slope': 0.8321861754078372}. Best is trial 0 with value: 0.0026881472253545585.
2025-08-29 18:00:47,863 - FilterSamples/Taylor: (train) mean of y values 1.0006264340729107.
2025-08-29 18:00:47,863 - FilterSamples/Taylor: (test) mean of y values 1.0022854508909003.
2025-08-29 18:00:48,134 - FilterSamples/Taylor: Selected feature 'FeatureTA_trend_ichimoku_b' with score 1.001704 and threshold 0.945817 (q=0.99).
2025-08-29 18:00:49,005 - FilterSamples/Taylor: Final score (train): 0.9978
2025-08-29 18:00:49,013 - FilterSamples/Taylor: Final score (test): 1.0353
2025-08-29 18:00:49,016 - FilterSamples/Taylor: Fraction of days with no coverage by test mask: 0.00%
2025-08-29 18:00:49,016 - Trial number 1
2025-08-29 18:00:49,858 -   Score (train) = 0.9978116357792318
2025-08-29 18:00:49,858 -   Score (test)

[I 2025-08-29 18:05:38,730] Trial 1 finished with value: 0.005923254233124162 and parameters: {'days_to_train_end': 35, 'taylor_horizon_days': 15, 'taylor_roll_window_days': 45, 'taylor_weight_slope': 0.15077135694841798}. Best is trial 1 with value: 0.005923254233124162.


...

2025-08-29 22:49:16,714 - Trial 56 finished with value: -0.00893358776350499 and parameters: {'days_to_train_end': 90, 'taylor_horizon_days': 70, 'taylor_roll_window_days': 15, 'taylor_weight_slope': 1.2434460756501369}. Best is trial 13 with value: 0.01060017053637097.
2025-08-29 22:49:30,283 - FilterSamples/Taylor: (train) mean of y values 1.0006264340729107.
2025-08-29 22:49:30,283 - FilterSamples/Taylor: (test) mean of y values 1.0022854508909003.
2025-08-29 22:49:30,637 - FilterSamples/Taylor: Selected feature 'FeatureTA_volatility_ui' with score 1.292486 and threshold 2.16392 (q=0.99).
2025-08-29 22:49:31,534 - FilterSamples/Taylor: Final score (train): 0.9965
2025-08-29 22:49:31,535 - FilterSamples/Taylor: Final score (test): 1.0252
2025-08-29 22:49:31,539 - FilterSamples/Taylor: Fraction of days with no coverage by test mask: 0.00%
2025-08-29 22:49:31,543 - Trial number 57
2025-08-29 22:49:32,406 -   Score (train) = 0.9964818135990926
2025-08-29 22:49:32,406 -   Score (test)  =

[I 2025-08-29 22:54:55,474] Trial 57 finished with value: 0.005704561394416668 and parameters: {'days_to_train_end': 45, 'taylor_horizon_days': 60, 'taylor_roll_window_days': 10, 'taylor_weight_slope': 1.1386130859049277}. Best is trial 13 with value: 0.01060017053637097.


2025-08-29 22:54:55,474 - Trial 57 finished with value: 0.005704561394416668 and parameters: {'days_to_train_end': 45, 'taylor_horizon_days': 60, 'taylor_roll_window_days': 10, 'taylor_weight_slope': 1.1386130859049277}. Best is trial 13 with value: 0.01060017053637097.
2025-08-29 22:55:09,433 - FilterSamples/Taylor: (train) mean of y values 1.0006264340729107.
2025-08-29 22:55:09,436 - FilterSamples/Taylor: (test) mean of y values 1.0022854508909003.
2025-08-29 22:55:09,742 - FilterSamples/Taylor: Selected feature 'FeatureTA_trend_ema_fast' with score 1.065511 and threshold 0.938739 (q=0.99).
2025-08-29 22:55:10,643 - FilterSamples/Taylor: Final score (train): 0.9986
2025-08-29 22:55:10,643 - FilterSamples/Taylor: Final score (test): 1.0009
2025-08-29 22:55:10,643 - FilterSamples/Taylor: Fraction of days with no coverage by test mask: 0.00%
2025-08-29 22:55:10,650 - Trial number 58
2025-08-29 22:55:11,390 -   Score (train) = 0.998578979047753
2025-08-29 22:55:11,392 -   Score (test)  

[I 2025-08-29 23:00:33,233] Trial 58 finished with value: 0.002861914217859884 and parameters: {'days_to_train_end': 30, 'taylor_horizon_days': 75, 'taylor_roll_window_days': 15, 'taylor_weight_slope': 1.0634043093824102}. Best is trial 13 with value: 0.01060017053637097.


2025-08-29 23:00:33,233 - Trial 58 finished with value: 0.002861914217859884 and parameters: {'days_to_train_end': 30, 'taylor_horizon_days': 75, 'taylor_roll_window_days': 15, 'taylor_weight_slope': 1.0634043093824102}. Best is trial 13 with value: 0.01060017053637097.
2025-08-29 23:00:33,238 - Best parameters: {'days_to_train_end': 115, 'taylor_horizon_days': 50, 'taylor_roll_window_days': 10, 'taylor_weight_slope': 1.268923120611385}
2025-08-29 23:00:33,243 - Best score: 0.01060017053637097
2025-08-29 23:00:33,259 - 
Trials DataFrame:
2025-08-29 23:00:33,273 -     number     value             datetime_start          datetime_complete               duration  params_days_to_train_end  params_taylor_horizon_days  params_taylor_roll_window_days  params_taylor_weight_slope     state
46      46 -0.009060 2025-08-29 21:51:38.306057 2025-08-29 21:56:48.312655 0 days 00:05:10.006598                        90                          75                              15                    1.017

In [10]:
df.to_parquet(f"{formatted_str}.parquet", index=False)